In [3]:
import dask
import os
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib

from letkf_forecasting import analyse_results, letkf_io

In [6]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)
base_folder = '/a2/uaren/travis'

# month_day = [[4, 9],  [4, 15], [4, 18],                                               
#              [5, 6],  [5, 9],  [5, 29],                                               
#              [6, 11], [6, 12],
             
#              [4, 2],  [4, 5],  [4, 19],                                               
#              [5, 7],  [5, 8],  [5, 19],                                               
#              [6, 3],  [6, 10], [6, 14],                                               
#              [6, 15],                                                                 
#              [4, 10], [4, 11], [4, 12],                                               
#              [4, 20], [4, 21], [4, 22],                                               
#              [4, 25], [4, 26], [5, 5],                                                
#              [5, 20], [5, 21], [5, 22],                                               
#              [5, 23], [5, 24], [5, 25],                                               
#              [5, 30], [6, 16], [6, 17],                                               
#              [6, 18], [6, 19], [6, 22]]

# month_day = [[4, 9],  [4, 18],                                               
#              [5, 6],  [5, 9],                                               
#              [6, 11], [6, 12],
             
#              [4, 2],  [4, 5],  [4, 19],                                               
#              [5, 7],  [5, 8],  [5, 19],                                               
#              [6, 3],  [6, 10], [6, 14],                                               
#              [6, 15],                                                                 
#              [4, 10], [4, 11], [4, 12],                                               
#              [4, 20], [4, 21], [4, 22],                                               
#              [4, 25], [5, 5],                                                
#              [5, 20], [5, 21], [5, 22],                                               
#              [5, 23], [5, 24], [5, 25],                                               
#              [5, 30], [6, 16], [6, 17],                                               
#              [6, 18], [6, 19], [6, 22]]


month_day = [[4, 15], [5, 29]]


year = 2014
# runs = ['opt_flow', 'wrf_no_div', 'owp_opt']
# runs = [['ensemble', 'owp_opt']]
# runs = [['anly_fore', 'owp_opt']]


# runs = ['persistence', 'opt_flow_with_div', 'opt_flow',
#         'wrf', 'wrf_no_div', 'owp_opt', 'radiosonde', 'wrf_mean']
# runs = ['persistence', 'opt_flow_with_div', 'opt_flow',
#         'wrf', 'wrf_no_div', 'owp_opt', 'radiosonde']
# runs = ['persistence', 'opt_flow', 'wrf_no_div', 'owp_opt',
#         ['ensemble', 'owp_opt']]
# runs = ['persistence', 'opt_flow', 'wrf_no_div', 'owp_opt']
# runs = [['ensemble', 'owp_opt']]

# runs = ['persistence', ['ensemble', 'owp_opt']]

# runs = ['radiosonde', 'wrf_mean', 'opt_flow',
#         'wrf_no_div', 'owp_opt',
#         ['anly_fore', 'owp_opt']]

runs = ['opt_flow', 'radiosonde', 'wrf_mean', 'persistence',
        'wrf_no_div', 'owp_opt',
        ['ensemble', 'owp_opt'],
        ['anly_fore', 'owp_opt']]

horizons = [15, 30, 45, 60]
only_cloudy = False
mean_win_size = None
only_of_times = True

one_km_err = True

ensemble_flag = False
analy_fore_flag = False
    
for amonth_day in month_day:
    print(amonth_day)
    month = amonth_day[0]
    day = amonth_day[1]
    dates = [datetime.date(2014, month, day)]

    error_stats_full = analyse_results.error_stats_many_days(
        dates, runs, horizons, base_folder,
        only_cloudy=only_cloudy, only_of_times=only_of_times,
        mean_win_size=mean_win_size, one_km_err=one_km_err
    )
    for error_stat in error_stats_full:
        if error_stat['name'][0] is 'anly_fore':
            analy_fore_flag = True
            run_name = error_stat['name'][1]
        elif error_stat['name'][0] is 'ensemble':
            ensemble_flag = True
            run_name = error_stat['name'][1]
        else:
            run_name = error_stat['name']
        results_folder_path = os.path.join(                                          
            base_folder,                                                               
            'results',                                                               
            f'{year:04}',                                                            
            f'{month:02}',                                                           
            f'{day:02}',                                                             
            run_name)
        results_folder_path = letkf_io.find_latest_run(
            results_folder_path)
        results_folder_path = os.path.join(
            results_folder_path, 'single_day')
        if not os.path.exists(results_folder_path):
            os.mkdir(results_folder_path)
        print(results_folder_path)
        for stat_name, stat in error_stat.items():
            if stat_name == 'name':
                continue
            if analy_fore_flag:
                stat_name = stat_name + '_anly_fore'
            elif ensemble_flag:
                stat_name = stat_name + '_ens'
            file_path = os.path.join(results_folder_path,
                                     f'{stat_name}.h5')
            stat.to_hdf(file_path, stat_name)
            print(stat_name)
        ensemble_flag = False
        analy_fore_flag = False

[4, 15]
opt_flow


/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg 

radiosonde


/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg 

wrf_mean


...

['ensemble', 'owp_opt']


/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg 

['anly_fore', 'owp_opt']


/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg == key:
/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/dask/core.py:306: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif type_arg is type(key) and arg 

/a2/uaren/travis/results/2014/05/29/opt_flow_001/single_day
rmse
forecast_sd
truth_sd
bias
correlation
stat_times
/a2/uaren/travis/results/2014/05/29/radiosonde_000/single_day
rmse
forecast_sd
truth_sd
bias
correlation
stat_times
/a2/uaren/travis/results/2014/05/29/wrf_mean_002/single_day
rmse
forecast_sd
truth_sd
bias
correlation
stat_times
/a2/uaren/travis/results/2014/05/29/persistence_000/single_day
rmse
forecast_sd
truth_sd
bias
correlation
stat_times
/a2/uaren/travis/results/2014/05/29/wrf_no_div_002/single_day
rmse
forecast_sd
truth_sd
bias
correlation
stat_times
/a2/uaren/travis/results/2014/05/29/owp_opt_000/single_day
rmse
forecast_sd
truth_sd
bias
correlation
stat_times
/a2/uaren/travis/results/2014/05/29/owp_opt_000/single_day
rmse_ens
forecast_sd_ens
truth_sd_ens
bias_ens
correlation_ens
stat_times_ens
/a2/uaren/travis/results/2014/05/29/owp_opt_000/single_day
rmse_anly_fore
forecast_sd_anly_fore
truth_sd_anly_fore
bias_anly_fore
correlation_anly_fore
stat_times_anly_fore
